### Курсовой проект

построение модели
для прогнозирования невыполнения долговых обязательств по текущему кредиту

#### Содержание проекта:

**Построение модели классификации**<br>
[1. Обзор обучающего датасета](#1)<br>
[2. Обработка выбросов](#2)<br>
[3. Обработка пропусков](#3)<br>
[4. Анализ данных](#4)<br>
[5. Отбор признаков](#5)<br>
[6. Балансировка классов](#6)<br>
[7. Подбор моделей, получение бейзлана](#7)<br>
[8. Выбор наилучшей модели, настройка гиперпараметров](#8)<br>
[9. Проверка качества, борьба с переобучением](#9)<br>
[10. Интерпретация результатов](#10)<br>

**Прогнозирование на тестовом датасете**<br>
[11. Выполнить для тестового датасета те же этапы обработки и постронияния признаков](#11)<br>
[12. Спрогнозировать целевую переменную, используя модель, построенную на обучающем датасете](#12)<br>

#### Практическая часть

In [1]:
# пути к директориям и файлам
TRAIN_DATASET_PATH = 'data/course_project_train.csv'
TEST_DATASET_PATH = 'data/course_project_test.csv'

In [2]:
# импорт необходимых библиотек
import numpy as np
import pandas as pd



##### Построение модели классификации

##### 1. Обзор обучающего датасета <a class="anchor" id="1"></a>

**Описание датасета**

* **Home Ownership** - домовладение
* **Annual Income** - годовой доход
* **Years in current job** - количество лет на текущем месте работы
* **Tax Liens** - налоговые обременения
* **Number of Open Accounts** - количество открытых счетов
* **Years of Credit History** - количество лет кредитной истории
* **Maximum Open Credit** - наибольший открытый кредит
* **Number of Credit Problems** - количество проблем с кредитом
* **Months since last delinquent** - количество месяцев с последней просрочки платежа
* **Bankruptcies** - банкротства
* **Purpose** - цель кредита
* **Term** - срок кредита
* **Current Loan Amount** - текущая сумма кредита
* **Current Credit Balance** - текущий кредитный баланс
* **Monthly Debt** - ежемесячный долг
* **Credit Default** - факт невыполнения кредитных обязательств (0 - погашен вовремя, 1 - просрочка)

In [3]:
# загрузим данные
df_train = pd.read_csv(TRAIN_DATASET_PATH)
df_test = pd.read_csv(TEST_DATASET_PATH)

In [4]:
df_train.head()

,Home Ownership,Annual Income,Years in current job,Tax Liens,Number of Open Accounts,Years of Credit History,Maximum Open Credit,Number of Credit Problems,Months since last delinquent,Bankruptcies,Purpose,Term,Current Loan Amount,Current Credit Balance,Monthly Debt,Credit Score,Credit Default
0,Own Home,482087.0,NaN,0.0,11.0,26.3,685960.0,1.0,NaN,1.0,debt consolidation,Short Term,99999999.0,47386.0,7914.0,749.0,0
1,Own Home,1025487.0,10+ years,0.0,15.0,15.3,1181730.0,0.0,NaN,0.0,debt consolidation,Long Term,264968.0,394972.0,18373.0,737.0,1
2,Home Mortgage,751412.0,8 years,0.0,11.0,35.0,1182434.0,0.0,NaN,0.0,debt consolidation,Short Term,99999999.0,308389.0,13651.0,742.0,0
3,Own Home,805068.0,6 years,0.0,8.0,22.5,147400.0,1.0,NaN,1.0,debt consolidation,Short Term,121396.0,95855.0,11338.0,694.0,0
4,Rent,776264.0,8 years,0.0,13.0,13.6,385836.0,1.0,NaN,0.0,debt consolidation,Short Term,125840.0,93309.0,7180.0,719.0,0


In [5]:
df_train.shape

(7500, 17)

In [6]:
df_train.iloc[0]

Home Ownership                            Own Home
Annual Income                               482087
Years in current job                           NaN
Tax Liens                                        0
Number of Open Accounts                         11
Years of Credit History                       26.3
Maximum Open Credit                         685960
Number of Credit Problems                        1
Months since last delinquent                   NaN
Bankruptcies                                     1
Purpose                         debt consolidation
Term                                    Short Term
Current Loan Amount                          1e+08
Current Credit Balance                       47386
Monthly Debt                                  7914
Credit Score                                   749
Credit Default                                   0
Name: 0, dtype: object

In [7]:
df_train['Credit Default'].value_counts()

0    5387
1    2113
Name: Credit Default, dtype: int64

##### 2. Обработка выбросов <a class="anchor" id="2"></a>

##### 3. Обработка пропусков <a class="anchor" id="3"></a>

##### 4. Анализ данных <a class="anchor" id="4"></a>

##### 5. Отбор признаков <a class="anchor" id="5"></a>

##### 6. Балансировка классов <a class="anchor" id="6"></a>

##### 7. Подбор моделей, получение бейзлана <a class="anchor" id="7"></a>

##### 8. Выбор наилучшей модели, настройка гиперпараметров <a class="anchor" id="8"></a>

##### 9. Проверка качества, борьба с переобучением <a class="anchor" id="9"></a>

##### 10. Интерпретация результатов <a class="anchor" id="10"></a>

##### Прогнозирование на тестовом датасете

##### 11. Выполнить для тестового датасета те же этапы обработки и построения признаков <a class="anchor" id="11"></a>

##### 12. Спрогнозировать целевую переменную, используя модель, построенную на обучающем датасете <a class="anchor" id="12"></a>